Mount Drive

In [ ]:
#do not run if you workspace is located somewhere else
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir("drive/MyDrive/Colab Notebooks")
os.listdir()
!ls

Mounted at /content/drive
 Assignment5.ipynb				  Posts.xml
 BM25_qrel_5.txt				  project_part1.ipynb
 BM25_result_5.txt				  __pycache__
 BM25_result.txt				  qrel.txt
 combMNZ.tsv					 'Question 4.ipynb'
 Entities					  RM3_result.txt
'IR_Project2_Nicholas Drummey, Ryan Reed.ipynb'  'run to qrel converter.ipynb'
 LGD_qrel_5.txt					  TF_IDF_qrel_5.txt
 LGD_result_5.txt				  TF_IDF_result_5.txt
 LGD_result.txt					  TF_IDF_result.txt
 pd_index					  Untitled0.ipynb
 post_parser_record.py				  word2vec.model
 Posts_Coffee.xml


Question 1######################################################################

In [ ]:
from post_parser_record import PostParserRecord
post_reader = PostParserRecord("Posts_Coffee.xml")
import nltk
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
from gensim.test.utils import datapath
from nltk.tokenize import word_tokenize
from scipy.spatial import distance
nltk.download('punkt')
import string
import re

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Word2Vec

In [ ]:



array = []
model = 0
sentance_simularity = {}

print("Indexing Questions...")
for question_id in post_reader.map_questions:
  question = post_reader.map_questions[question_id]
  title = question.title.lower().strip()
  split_text = word_tokenize(re.sub(r'[^\w\s]', '', title))

  array.append(split_text)
print("Complete")


model = Word2Vec(sentences=array, window=5, min_count=1, workers=4)
model.save("word2vec.model")

# train the word2vec model with the question titles
model = Word2Vec.load("word2vec.model")
model.train(array, total_examples=len(array), epochs=10)


# testing vector adding and dividing and such
vector1 = model.wv['caffeine']
vector2 = model.wv['tool']

query_vector = (model.wv['when']+model.wv['does']+model.wv['coffee']+model.wv['go']+model.wv['off'])
query_vector = query_vector/5
 
for sentence in array:
  sentence_vector = 0
  for word in sentence:
    sentence_vector += model.wv[word]
  sentence_vector = sentence_vector/len(sentence)
  sentance_simularity[str(sentence)] = 1-distance.cosine(query_vector, sentence_vector)


result = sorted(sentance_simularity.items(), key=lambda x: x[1], reverse=True)

i = 0
for items in result:
  #makes sure only 5 of each are printed
  if i <= 4:
    if items[1] == 1:
      print("Score: "+str(items[1])+"\t\t\tTitle: "+str(items[0]))
    else:
      print("Score: "+str(items[1])+"\tTitle: "+str(items[0]))
    i += 1
  else:
    break



Indexing Questions...
Complete


Score: 1			Title: ['when', 'does', 'coffee', 'go', 'off']
Score: 0.9999874234199524	Title: ['why', 'does', 'instant', 'coffee', 'foam', 'when', 'stirring', 'it']
Score: 0.9999862313270569	Title: ['what', 'am', 'i', 'supposed', 'to', 'do', 'when', 'coffee', 'shop', 'leave', 'some', 'coffee', 'beans', 'in', 'my', 'latte']
Score: 0.9999860525131226	Title: ['does', 'sweet', 'coffee', 'exist']
Score: 0.9999857544898987	Title: ['does', 'coffee', 'taste', 'different', 'when', 'it', 'is', 'ground', 'at', 'home']


Question 2######################################################################

In [ ]:
!pip install python-terrier
!pip install --upgrade git+https://github.com/terrier-org/pyterrier.git
import pyterrier as pt
pt.init(boot_packages=["com.github.terrierteam:terrier-prf:-SNAPSHOT"])

import pandas as pd
!rm -rf ./pd_index
pd_indexer = pt.DFIndexer("./pd_index")

import nltk
nltk.download('punkt')
nltk.download('stopwords')

In [ ]:
docno_list = []
url_list = []
text_list = []
#indexed questions
print("Indexing Questions...")
for question_id in post_reader.map_questions:
  question = post_reader.map_questions[question_id]
  text = question.body.lower().strip()
  docno_list.append(str(question_id))
  url_list.append("url"+str(question_id))
  text_list.append(text)
print("Complete")

print("Finalizing...")

#adds all collected data to a DataFrame from above
df = pd.DataFrame({'docno':docno_list, 'url':url_list, 'text':text_list})
indexref = pd_indexer.index(df["text"], df)
index = pt.IndexFactory.of(indexref)

#prints info on colected documents, (should have 185140 documents)
print(index.getCollectionStatistics().toString())

Indexing Questions...
Complete
Finalizing...
Number of documents: 1370
Number of terms: 5225
Number of postings: 43420
Number of fields: 0
Number of tokens: 57129
Field names: []
Positions:   false



In [ ]:
import copy
def searchTopK(k):
  querys = ["1","how to make espresso"], ["2", "moka pot"], ["3", "coffee caffeine"]
  #add querys to dataframe
  queries = pd.DataFrame(querys,columns=['qid','query'])
  #find top 1000 results with pyterrier Batch Retrieve, from the created index
  result = pt.BatchRetrieve(indexref, num_results =k, wmodel="TF_IDF").transform(queries)
  #write results to file and print
  #print(result)
  last_query = ''

  tag_frequency = {}
  passes_num = 0
  for i in result.docno:
    question = post_reader.map_questions[int(i)]
    #print benining
    if last_query is not result.qid[passes_num]:
      print("Query: "+str(querys[int(result.qid[passes_num])-1][1]))
      tag_frequency = {}
    last_query = copy.copy(result.qid[passes_num])

  
    passes_num+=1
    #print(question.tags)
    for tag in question.tags:
      if tag not in tag_frequency:
        tag_frequency[tag] = 1
      else:
        tag_frequency[tag] += 1
      
    #print ending
    if (int(passes_num/k+1)) is not int(result.qid[passes_num-1]):
      sorted_tags = sorted(tag_frequency.items(), key=lambda x: x[1], reverse=True)
      print("most common tag is: ["+str(sorted_tags[0][0])+"] with frequency of: "+str(sorted_tags[0][1]))

print("Tag of top 1 result:")
searchTopK(1)
print("\nTag of top 5 result:")
searchTopK(5)
print("\nTag of top 10 result:")
searchTopK(10)
print("\nTag of top 20 result:")
searchTopK(20)

Tag of top 1 result:
Query: how to make espresso
most common tag is: [espresso] with frequency of: 1
Query: moka pot
most common tag is: [equipment] with frequency of: 1
Query: coffee caffeine
most common tag is: [decaffeinated] with frequency of: 1

Tag of top 5 result:
Query: how to make espresso
most common tag is: [espresso] with frequency of: 5
Query: moka pot
most common tag is: [moka] with frequency of: 5
Query: coffee caffeine
most common tag is: [caffeine] with frequency of: 4

Tag of top 10 result:
Query: how to make espresso
most common tag is: [espresso] with frequency of: 8
Query: moka pot
most common tag is: [moka] with frequency of: 10
Query: coffee caffeine
most common tag is: [caffeine] with frequency of: 8

Tag of top 20 result:
Query: how to make espresso
most common tag is: [espresso] with frequency of: 17
Query: moka pot
most common tag is: [moka] with frequency of: 19
Query: coffee caffeine
most common tag is: [caffeine] with frequency of: 18
